[진행 상황]
1. 동영상을 재생하여 프레임 저장
 - 프레임 한 개를 불러와 마우스로 x,y,w,h 좌표를 추출
2. 좌표를 통해 roi 설정
2. 동영상 frame을 gray scale 후 roi 영역에는 binary scale 추가
4. 1, 2, 3루의 중앙 좌표 설정
      - roi 내 누상의 중앙 좌표는 모든 방송사가 거의 같기 때문에 바꾸지 않아도 됨 
3. <조건>
 - 피칭 장면일 때만 True
 - 미리 지정한 사진 (누상 사진)과 이미지 템플릿 매칭
      - roi 영역 내 누상만 검출하기 위해서
 - 이전 frame과 현재 프레임 비교
      - "1루에서 2루" 등 누상이 바뀌는 부분 검출하기 위해서
4. 다음 피칭 시점에서 누상 바뀐 frame 폴더에 저장

[문제] 
 - OUT과 점수가 난 상황을 감지 못함

[목표]
1. 다음 피칭 시점에서 현재 시점으로 바꾸기
2. 논문 확장 시 : roi 좌표를 입력하지 않고 검출할 수 있게 하기 (모델 학습)
   - 효과 : 누상이 없는 부분을 검출하지 않음
3. 점수가 났을 때와 OUT 됐을 때 구분하기

In [1]:
import os
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 초기화 할 GPU Number
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.7
sess = tf.compat.v1.Session(config=config)

from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import pickle
import pandas as pd
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
model = load_model('./paper_data')
classes = os.listdir('./paper_dataset/train')

In [3]:
# 스코어보드 이미지 템플릿 매칭

def template_matching(biimg, template): # src: 들어갈 이미지 또는 프레임, template: 템플릿 이미지
    flag = False
    dst = biimg.copy()
#     dst = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
#     _, template = cv2.threshold(template, 127, 255, cv2.THRESH_BINARY_INV)
    src = cv2.cvtColor(biimg, cv2.COLOR_BGR2GRAY)
    template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    result = cv2.matchTemplate(src, template, cv2.TM_SQDIFF_NORMED)
#     result2 = cv2.matchTemplate(dst, template, cv2.TM_CCORR)

    threshold = 0.08
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(result)
        
#     x, y = minLoc
#     h, w = template.shape[:-1]
    print(minVal)
    if minVal < threshold:
        flag = True
    
    return flag

In [4]:
# 직전 프레임과 현재 프레임 비교

def prev_frame(biimg, count):         
    global nowFrame   
    global preFrame
       
    preFrame_1 = []
    preFrame_2 = []
    nowFrame_1 = []
    nowFrame_2 = []
    
    compare_Frame = []
       
    if (count % 2 == 1):
# 현재 프레임의 1, 2, 3루의 RGB값 3x3 배열로 저장
        nowFrame = []
        nowFrame_1 = np.vstack([biimg[25][38],biimg[20][27]])
        
        nowFrame_2 = np.vstack([nowFrame_1,biimg[23][10]])
        nowFrame = nowFrame_2
        
    else:
# 3개 전 프레임의 1, 2, 3루의 RGB값 3x3 배열로 저장
        preFrame = []
        preFrame_1 = np.vstack([biimg[25][38],biimg[20][27]])
        preFrame_2 = np.vstack([preFrame_1,biimg[23][10]])
        preFrame = preFrame_2
        
# 이전 프레임과 현재 프레임 비교
    if np.array_equal(preFrame, nowFrame):
        compareFrame = False
    else:
        compareFrame = True

    return compareFrame

In [5]:
# main

cap = cv2.VideoCapture('C:/Users/user/data123/mbc_NCvs두산/mbc_NCvs두산/NCvs두산풀경기.mp4')
template = cv2.imread('./0roo/1(3).jpg', cv2.COLOR_BGR2GRAY)
curTime = time.time()
nowFrame = []
preFrame = []
x=136; y=139; w=187; h=179
count = 0

# 원하는 프레임부터 동영상 재생
# cap.set(cv2.CAP_PROP_POS_FRAMES, 40200)

while cv2.VideoCapture():
     
    ret, frame = cap.read()
    gimoon = frame.copy()
    roi = frame[y:h, x:w]
    _, biimg = cv2.threshold(roi, 127, 255, cv2.THRESH_BINARY_INV)
    cv2.rectangle(gimoon, (x,y), (w-1, h-1), (0,255,0), 1)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
   
    frame = cv2.resize(frame.astype('float32') / 255, (224, 224))
    predict = model.predict(frame[np.newaxis]) #np.expand_dims(frame, axis=0))
    label = 'not pitching' if np.argmax(predict) == 0 else 'pitching'
    
    cv2.imshow('frame',gimoon)
    if label == 'pitching':
        if template_matching(gimoon, template):
            print(count)
            nusang = ""
            base_1 = np.array_equal(biimg[25][38], [255,0,0]) # 1루 False: 변화 X, True: 변화 O
            base_2 = np.array_equal(biimg[10][27], [255,0,0]) # 2루 False: 변화 X, True: 변화 O
            base_3 = np.array_equal(biimg[23][10], [255,0,0]) # 3루 False: 변화 X, True: 변화 O
            compareFrame = prev_frame(biimg, count)

            if compareFrame:
                if base_1:
                    nusang += "1루 변화 "
                if base_2:
                    nusang += "2루 변화 "  
                if base_3:
                    nusang += "3루 변화 "        

            print(nusang,"\n")
            if len(nusang) != 0:
                cv2.imwrite("./frame/frame_{:06d}.jpg".format(count), biimg)
    if cv2.waitKey(1) &0xFF == ord('q'):
        break

    count += 1    
    
cap.release()
cv2.destroyAllWindows() 

0.046039123088121414
7937
 

0.04609134793281555
7938
 

0.04569561779499054
7939
 

0.046305567026138306
7940
 

0.04625499248504639
7941
 

0.04570215567946434
7942
 

0.04655444622039795
7943
 

0.046914149075746536
7944
 

0.04723908752202988
7945
 

0.05144638195633888
7951
 

0.05592095106840134
7952
 

0.06242063641548157
7953
 

0.06775791198015213
7954
 

0.06977363675832748
7955
 

0.0674041211605072
7956
 

0.06284289807081223
7957
 

0.05578118935227394
7958
 

0.05141826346516609
7959
 

0.04907745495438576
7960
 

0.049689944833517075
7961
 

0.04917224124073982
7962
 

0.048253659158945084
7963
 

0.04728405177593231
7964
 

0.0481378510594368
7965
 

0.04804672300815582
7966
 

0.04313155636191368
7967
 

0.040107160806655884
7968
 

0.0397324338555336
7969
 

0.05390956997871399
7970
 

0.03956746682524681
7971
 

0.03555185720324516
7972
 

0.034227289259433746
7973
 

0.03368803486227989
7974
 

0.031608715653419495
7975
 

0.030448125675320625
7976
 

0.030913727357

0.014511979185044765
29269
 

0.01451224647462368
29270
 

0.014511979185044765
29271
 

0.01451224647462368
29272
 

0.01451237965375185
29273
 

0.014511979185044765
29274
 

0.014511979185044765
29275
 

0.01451224647462368
29278
 

0.014511979185044765
29279
 

0.014512130059301853
29280
 

0.014511862769722939
29281
 

0.014512130059301853
29282
 

0.014512397348880768
29283
 

0.014511996880173683
29284
 

0.01508024986833334
29387
 

0.015147139318287373
29654
 

0.015256895683705807
29687
 

0.01526704989373684
29688
 

0.015267672948539257
29689
 

0.015265428461134434
29694
 

0.015189087018370628
29696
 

0.015187271870672703
29697
 

0.015185303054749966
29698
 

0.015194377861917019
29699
 

0.015173601917922497
29702
 

0.015174003317952156
29704
 

0.015173736028373241
29705
 

0.015173468738794327
29706
 

0.015174269676208496
29707
 

0.015173736028373241
29708
 

0.015177455730736256
29709
 

0.015174850821495056
29710
 

0.015212330035865307
29711
 

0.01521233003586

0.017087064683437347
32188
 

0.01704031601548195
32189
 

0.01710810884833336
32190
 

0.017106134444475174
32191
 

0.01711002178490162
32192
 

0.017106831073760986
32194
 

0.017104020342230797
32200
 

0.017103884369134903
32201
 

0.017103884369134903
32202
 

0.01711839623749256
32203
 

0.013975640758872032
32204
 

0.011606331914663315
32205
 

0.011305274441838264
32206
 

0.01291784830391407
32207
 

0.013895557262003422
32208
 

0.015554452314972878
32210
 

0.015192490071058273
32211
 

0.01710917055606842
32212
 

0.017108900472521782
32215
 

0.01710917055606842
32216
 

0.0170816108584404
32217
 

0.01704382337629795
32218
 

0.016981760039925575
32219
 

0.01686180755496025
32220
 

0.01667848415672779
33569
 

0.016678886488080025
33570
 

0.016678886488080025
33571
 

0.016678886488080025
33572
 

0.01667902246117592
33573
 

0.01668117754161358
33579
 

0.016680371016263962
33580
 

0.01668063923716545
33581
 

0.016631724312901497
33582
 

0.013999397866427898
3358

0.011611148715019226
36839
 

0.011488002724945545
36840
 

0.011485716328024864
36841
 

0.011490925215184689
36842
 

0.01148717850446701
36843
 

0.01148800365626812
36844
 

0.011492897756397724
36845
 

0.01149427704513073
36846
 

0.01149427704513073
36847
 

0.01149427704513073
36848
 

0.01149455551058054
36849
 

0.011494695208966732
36850
 

0.01149455551058054
36851
 

0.01149455551058054
36852
 

0.011494416743516922
36853
 

0.01149455551058054
36854
 

0.01149455551058054
36855
 

0.01149455551058054
36856
 

0.01149427704513073
36857
 

0.01149427704513073
36858
 

0.011540413834154606
37825
 

0.01154069323092699
37827
 

0.011540274135768414
37828
 

0.011540274135768414
37829
 

0.011536375619471073
37830
 

0.011550885625183582
37831
 

0.011561213061213493
37832
 

0.01158552709966898
37833
 

0.01158617902547121
37834
 

0.011584916152060032
37835
 

0.011580965481698513
37836
 

0.01158172357827425
37837
 

0.011584953404963017
37838
 

0.011586369946599007
37839


0.014897391200065613
45181
 

0.015286307781934738
45182
 

0.014035087078809738
45183
 

0.01599826104938984
45184
 

0.016002563759684563
45185
 

0.015995224937796593
45186
 

0.015995224937796593
45187
 

0.015995364636182785
45188
 

0.01600695215165615
45189
 

0.0160016231238842
45190
 

0.01603233814239502
45191
 

0.01603284664452076
45192
 

0.016029877588152885
45193
 

0.016031889244914055
45194
 

0.016036592423915863
45195
 

0.016036175191402435
45196
 

0.01603645458817482
45197
 

0.016036592423915863
45198
 

0.016036175191402435
45199
 

0.016036314889788628
45200
 

0.016048364341259003
45201
 

0.016057860106229782
45202
 

0.016057796776294708
45203
 

0.016059542074799538
45204
 

0.01605583354830742
45205
 

0.0159994438290596
45206
 

0.01599680818617344
45207
 

0.01602960377931595
45208
 

0.016033809632062912
45209
 

0.016066331416368484
45210
 

0.01613236777484417
45211
 

0.01614716462790966
45213
 

0.016147304326295853
45214
 

0.016147444024682045
452

...

0.016602773219347
312533
 

0.016601409763097763
312535
 

0.016593044623732567
312537
 

0.016586052253842354
312538
 

0.0165848471224308
312539
 

0.01654946058988571
312540
 

0.016518108546733856
312541
 

0.01653711125254631
312542
 

0.01653711125254631
312543
 

0.016515612602233887
312544
 

0.016519242897629738
312545
 

0.016517626121640205
312546
 

0.016518129035830498
312547
 

0.016517993062734604
312548
 

0.01651785895228386
312549
 

0.016517722979187965
312550
 

0.01651785895228386
312551
 

0.01652536913752556
312552
 

0.016536669805645943
312553
 

0.016536112874746323
312554
 

0.016535978764295578
312555
 

0.016536248847842216
312556
 

0.016536248847842216
312557
 

0.01653638295829296
312558
 

0.016536518931388855
312559
 

0.016595464199781418
312560
 

0.016535857692360878
312561
 

0.016623158007860184
312562
 

0.016615817323327065
312563
 

0.016614671796560287
312564
 

0.016614941880106926
312565
 

0.01671077124774456
312566
 

0.016749180853366852


0.014910437166690826
318746
 

0.014910838566720486
318747
 

0.014910838566720486
318748
 

0.014910571277141571
318749
 

0.014912831597030163
318750
 

0.014918247237801552
318751
 

0.014926118776202202
318752
 

0.01492602564394474
318753
 

0.014917384833097458
318754
 

0.01492541003972292
318755
 

0.014924874529242516
318756
 

0.014925142750144005
318757
 

0.014925142750144005
318758
 

0.01492500863969326
318759
 

0.014925142750144005
318760
 

0.014925142750144005
318761
 

0.014924874529242516
318762
 

0.014925142750144005
318763
 

0.01492500863969326
318764
 

0.014925142750144005
318765
 

0.014940868131816387
318766
 

0.014868262223899364
318767
 

0.014929763041436672
318768
 

0.014923593029379845
318769
 

0.015139986760914326
318793
 

0.01514012087136507
318795
 

0.016375232487916946
319170
 

0.016390269622206688
319172
 

0.01639353111386299
319173
 

0.01645950973033905
319174
 

0.01645950973033905
319175
 

0.01645950973033905
319176
 

0.016459509730339

0.011581383645534515
323603
 

0.011574847623705864
323604
 

0.011574986390769482
323605
 

0.011574847623705864
323606
 

0.011574847623705864
323607
 

0.011574847623705864
323608
 

0.011575610376894474
323609
 

0.011576204560697079
323610
 

0.011592501774430275
323611
 

0.011592780239880085
323612
 

0.011590330861508846
323613
 

0.011590469628572464
323614
 

0.011590469628572464
323615
 

0.011590330861508846
323616
 

0.011608083732426167
323617
 

0.01161646656692028
323618
 

0.011615021154284477
323619
 

0.01161336712539196
323620
 

0.01161317527294159
323621
 

0.011609754525125027
323622
 

0.011613487266004086
323623
 

0.01159345917403698
323624
 

0.011577597819268703
323625
 

0.011590118519961834
323626
 

0.011590396985411644
323627
 

0.011590118519961834
323628
 

0.011591149494051933
323629
 

0.011590871028602123
323630
 

0.011590871028602123
323631
 

0.011591009795665741
323632
 

0.011590732261538506
323633
 

0.011591149494051933
323634
 

0.0115910097

0.014599097892642021
328654
 

0.014583651907742023
328655
 

0.014525704085826874
328656
 

0.014550594612956047
328657
 

0.014565455727279186
328658
 

0.01454786118119955
328659
 

0.01456553116440773
328660
 

0.014558473601937294
328661
 

0.0145771699026227
328662
 

0.014583763666450977
328663
 

0.014581389725208282
328664
 

0.014586856588721275
328665
 

0.014586994424462318
328666
 

0.014586994424462318
328667
 

0.014586856588721275
328668
 

0.014586856588721275
328669
 

0.014586717821657658
328670
 

0.014586856588721275
328671
 

0.014587133191525936
328672
 

0.014586856588721275
328673
 

0.014587133191525936
328674
 

0.014587133191525936
328675
 

0.014586994424462318
328676
 

0.014586994424462318
328677
 

0.014587133191525936
328678
 

0.01458657905459404
328679
 

0.014631424099206924
328680
 

0.014631700702011585
328681
 

0.014631561934947968
328682
 

0.014631839469075203
328683
 

0.014622922986745834
328684
 

0.014601118862628937
328685
 

0.01458925474

0.014475875534117222
329650
 

0.014476394280791283
329651
 

0.014480097219347954
329652
 

0.014490025117993355
329653
 

0.014493529684841633
329654
 

0.014508789405226707
329655
 

0.014533053152263165
329656
 

0.014533329755067825
329657
 

0.014532914385199547
329658
 

0.014533190988004208
329659
 

0.014533468522131443
329660
 

0.014533190988004208
329661
 

0.014532914385199547
329662
 

0.014533053152263165
329663
 

0.014533190988004208
329664
 

0.014533190988004208
329665
 

0.014533053152263165
329666
 

0.014532914385199547
329667
 

0.014533053152263165
329668
 

0.014533053152263165
329669
 

0.014533329755067825
329670
 

0.014508901163935661
329671
 

0.014529822394251823
329672
 

0.014528559520840645
329673
 

0.014538330025970936
329674
 

0.014546730555593967
329675
 

0.01456701010465622
329676
 

0.014563728123903275
329677
 

0.014552952721714973
329678
 

0.014540054835379124
329679
 

0.014547761529684067
329680
 

0.014547208324074745
329681
 

0.0145476

## 동영상 재생하여 좌표 추출하기

In [6]:
# cap1 = cv2.VideoCapture('C:/Users/user/data123/mbc_NCvs두산/mbc_NCvs두산/NCvs두산풀경기.mp4')
# count = 0

# # 원하는 프레임부터 동영상 재생
# cap1.set(cv2.CAP_PROP_POS_FRAMES, 40200)

# while cv2.VideoCapture():
     
#     ret, frame = cap1.read()
#     cv2.imshow("frame",frame)
#     cv2.imwrite("./save_frame/frame_{:06d}.jpg".format(count), frame)
#     if cv2.waitKey(1) &0xFF == ord('q'):
#         break

#     count += 1   

# cap1.release()
# cv2.destroyAllWindows() 

In [7]:
# # import cv2
# # import numpy as np

# isDragging = False
# x0,y0,w,h = -1,-1,-1,-1
# blue,red = (255,0,0),(0,0,255)

# def onMouse(event, x,y,flags, param):
#     global isDragging,x0,y0,img
#     if event ==cv2.EVENT_LBUTTONDOWN:
#         isDragging=True
#         x0=x
#         y0=y
#     elif event==cv2.EVENT_MOUSEMOVE:
#         if isDragging:
#             img_draw=img.copy()
#             cv2.rectangle(img_draw,(x0,y0),(x,y),blue,1)
#             cv2.imshow('img',img_draw)
#     elif event== cv2.EVENT_LBUTTONUP:
#         if isDragging:
#             isDragging=False
#             yet_w=x-x0
#             yet_h=y-y0
#             w = yet_w + x0
#             h = yet_h + y0
#             print('x:%d, y%d,w:%d,h:%d'%(x0,y0,w,h))
#             if w>0 and h>0:
#                 img_draw= img.copy()
#                 cv2.rectangle(img_draw,(x0,y0),(x,y),red,1)
#                 cv2.imshow('img',img_draw)
#                 roi = img[y0:y0+h,x0:x0+w]
#                 cv2.imshow('cropped',roi)
#                 cv2.moveWindow('cropped',0,0)
#                 cv2.imwrite('./cropped.jpg',roi)
#                 print('cropped')
#             else:
#                 cv2.imshow('img',img)
#                 print("좌측상단에서 우측하단으로 영역을 드래그하세요")
# img=cv2.imread('./save_frame/frame_000000.jpg')
# cv2.imshow('img',img)
# cv2.setMouseCallback('img',onMouse)
# cv2.waitKey()
# cv2.destroyAllWindows()